In [81]:
import pandas as pd
import pathlib
import os
import xlrd
import re
from pathlib import Path

### F.1 Peptigram
#### Formats the excel files into the format of Peptigram input. 
##### Change protein = '***' to the short protein accession number of the protein you wish to make a peptigram of

In [6]:
protein = 'P68871'
files = os.listdir('protein-peptide_sample_inf')
files_xls = [f for f in files if f[-3:] == 'lsx']
df_inf = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel('protein-peptide_sample_inf' + '/' + f)
    data = data.loc[(data['#Feature'] >= 4)]
    data['Peptide']= data['Peptide'].str.replace('\d+', '')
    data['Peptide']= data['Peptide'].str.replace(r'\(\+\.\)', '')
    data['Peptide']= data['Peptide'].apply(lambda x: x.split('.')[1] if x[1]=='.' else x.split('.')[0] if (x[1]!='.' and x[-2]=='.') else x)
    data['count'] = 1
    data = data.groupby(by=['Peptide', 'Protein Accession'], as_index=False).sum()
    data['End'] = data['End']/ data['count']
    data['Start'] = data['Start']/ data['count']
    df_inf = df_inf.append(data)
    
df_inf['count']=1
df_inf.reset_index(inplace=True, drop=True)

files = os.listdir('protein-peptide_sample_ninf')
files_xls = [f for f in files if f[-3:] == 'lsx']
df_ninf = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel('protein-peptide_sample_ninf' + '/' + f)
    data = data.loc[(data['#Feature'] >= 4)]
    data['Peptide']= data['Peptide'].str.replace('\d+', '')
    data['Peptide']= data['Peptide'].str.replace(r'\(\+\.\)', '')
    data['Peptide']= data['Peptide'].apply(lambda x: x.split('.')[1] if x[1]=='.' else x.split('.')[0] if (x[1]!='.' and x[-2]=='.') else x)
    data['count'] = 1
    data = data.groupby(by=['Peptide', 'Protein Accession'], as_index=False).sum()
    data['End'] = data['End']/ data['count']
    data['Start'] = data['Start']/ data['count']
    df_ninf = df_ninf.append(data)
    
df_ninf['count']=1
df_ninf.reset_index(inplace=True, drop=True)

files = os.listdir('protein-peptide_sample_wf')
files_xls = [f for f in files if f[-3:] == 'lsx']
df_WF = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel('protein-peptide_sample_wf' + '/' + f)
    data = data.loc[(data['#Feature'] >= 4)]
    data['Peptide']= data['Peptide'].str.replace('\d+', '')
    data['Peptide']= data['Peptide'].str.replace(r'\(\+\.\)', '')
    data['Peptide']= data['Peptide'].apply(lambda x: x.split('.')[1] if x[1]=='.' else x.split('.')[0] if (x[1]!='.' and x[-2]=='.') else x)
    data['count']=1
    data = data.groupby(by=['Peptide', 'Protein Accession'], as_index=False).sum()
    data['End'] = data['End']/ data['count']
    data['Start'] = data['Start']/ data['count']
    df_WF = df_WF.append(data)
    
df_WF['count']=1
df_WF.reset_index(inplace=True, drop=True)

df_inf = df_inf[['Protein Accession', 'Peptide', '#Feature', 'Start', 'End', 'count']]
df_inf = df_inf.groupby(by=['Peptide', 'Protein Accession'], as_index=False).sum()
df_inf['End'] = df_inf['End']/ df_inf['count']
df_inf['Start'] = df_inf['Start']/ df_inf['count']
df_inf[['End', 'Start']] = df_inf[['End', 'Start']].round(decimals=0)
df_inf['#Feature'] = df_inf['#Feature']/ 3
df_inf['Protein Accession'] = df_inf['Protein Accession'].map(lambda x: str(x[3:]) )
df_inf['Protein Accession'] = df_inf['Protein Accession'].str[:6]
df_inf = df_inf[['Peptide', 'Protein Accession', 'Start', 'End', '#Feature']]
df_inf.sort_values(['#Feature'], ascending=False)
df_temp = df_inf
df_temp = df_inf.loc[df_inf['Protein Accession'] == protein]
df_temp.rename(columns={'Peptide' : 'Peptide', "Protein Accession":"UniProt id" ,"Start" : "Start position", 'End': 'End position', '#Feature' : 'Intensity 1'}, inplace=True)
print(df_temp)

df_ninf = df_ninf[['Protein Accession', 'Peptide', '#Feature', 'Start', 'End', 'count']]
df_ninf['Start'] =df_ninf['Start'].apply(lambda x: int(x))
df_ninf['End'] =df_ninf['End'].apply(lambda x: int(x))
df_ninf = df_ninf.groupby(by=['Peptide', 'Protein Accession'], as_index=False).sum()
df_ninf['Start'] =df_ninf['Start'].apply(lambda x: int(x))
df_ninf['End'] =df_ninf['End'].apply(lambda x: int(x))
df_ninf['End'] = df_ninf['End']/ df_ninf['count']
df_ninf['Start'] = df_ninf['Start']/ df_ninf['count']
df_ninf[['End', 'Start']] = df_ninf[['End', 'Start']].round(decimals=0)
df_ninf['#Feature'] = df_ninf['#Feature']/ 3
df_ninf['Protein Accession'] = df_ninf['Protein Accession'].map(lambda x: str(x[3:]) )
df_ninf['Protein Accession'] = df_ninf['Protein Accession'].str[:6]
df_ninf = df_ninf[['Peptide', 'Protein Accession', 'Start', 'End', '#Feature']]
df_ninf.sort_values(['#Feature'], ascending=False)
df_temp = df_ninf
df_temp = df_ninf.loc[df_ninf['Protein Accession'] == protein]
df_temp.rename(columns={'Peptide' : 'Peptide', "Protein Accession":"UniProt id" ,"Start" : "Start position", 'End': 'End position', '#Feature' : 'Intensity 1'}, inplace=True)
print(df_temp)

df_WF = df_WF[['Protein Accession', 'Peptide', '#Feature', 'Start', 'End', 'count']]
df_WF['Start'] =df_WF['Start'].apply(lambda x: int(x))
df_WF['End'] =df_WF['End'].apply(lambda x: int(x))
df_WF = df_WF.groupby(by=['Peptide', 'Protein Accession'], as_index=False).sum()
df_WF['Start'] =df_WF['Start'].apply(lambda x: int(x))
df_WF['End'] =df_WF['End'].apply(lambda x: int(x))
df_WF['End'] = df_WF['End']/ df_WF['count']
df_WF['Start'] = df_WF['Start']/ df_WF['count']
df_WF[['End', 'Start']] = df_WF[['End', 'Start']].round(decimals=0)
df_WF['#Feature'] = df_WF['#Feature']/ 5
df_WF['Protein Accession'] = df_WF['Protein Accession'].map(lambda x: str(x[3:]) )
df_WF['Protein Accession'] = df_WF['Protein Accession'].str[:6]
df_WF = df_WF[['Peptide', 'Protein Accession', 'Start', 'End', '#Feature']]
df_WF.sort_values(['#Feature'], ascending=False)
df_temp = df_WF
df_temp = df_WF.loc[df_WF['Protein Accession'] == protein]
df_temp.rename(columns={'Peptide' : 'Peptide', "Protein Accession":"UniProt id" ,"Start" : "Start position", 'End': 'End position', '#Feature' : 'Intensity 1'}, inplace=True)
print(df_temp)

                  Peptide UniProt id  Start position  End position  \
25    AAYQKVVAGVANALAHKYH     P68871           129.0         147.0   
52               AFSDGLAH     P68871            71.0          78.0   
54              AFSDGLAHL     P68871            71.0          79.0   
56          AFSDGLAHLDNLK     P68871            71.0          83.0   
57      AFSDGLAHLDNLKGTFA     P68871            71.0          87.0   
...                   ...        ...             ...           ...   
1844           YQKVVAGVAN     P68871           131.0         140.0   
1846          YQKVVAGVANA     P68871           131.0         141.0   
1848         YQKVVAGVANAL     P68871           131.0         142.0   
1850        YQKVVAGVANALA     P68871           131.0         143.0   
1852    YQKVVAGVANALAHKYH     P68871           131.0         147.0   

      Intensity 1  
25       2.666667  
52       1.333333  
54       1.333333  
56       1.666667  
57       1.333333  
...           ...  
1844     1.333333  

C:\Users\Kalle\anaconda3\envs\pandas2\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\Kalle\anaconda3\envs\pandas2\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\Kalle\anaconda3\envs\pandas2\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return

### F.2 Proteasix
#### F.2.1 Formats the difference excel files into the format of proteasix input. 

In [159]:
files = os.listdir('protein-peptide_sample_inf')
files_xls = [f for f in files if f[-3:] == 'lsx']
df_inf = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel('protein-peptide_sample_inf' + '/' + f)
    data = data.loc[(data['#Feature'] >= 4)]
    data['Peptide']= data['Peptide'].str.replace('\d+', '')
    data['Peptide']= data['Peptide'].str.replace(r'\(\+\.\)', '')
    data['Peptide']= data['Peptide'].apply(lambda x: x.split('.')[1] if x[1]=='.' else x.split('.')[0] if (x[1]!='.' and x[-2]=='.') else x)
    data['count'] = 1
    data = data.groupby(by=['Peptide', 'Protein Accession'], as_index=False).sum()
    data['End'] = data['End']/ data['count']
    data['Start'] = data['Start']/ data['count']
    df_inf = df_inf.append(data)
    
df_inf['count']=1
df_inf.reset_index(inplace=True, drop=True)

files = os.listdir('protein-peptide_sample_ninf')
files_xls = [f for f in files if f[-3:] == 'lsx']
df_ninf = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel('protein-peptide_sample_ninf' + '/' + f)
    data = data.loc[(data['#Feature'] >= 4)]
    data['Peptide']= data['Peptide'].str.replace('\d+', '')
    data['Peptide']= data['Peptide'].str.replace(r'\(\+\.\)', '')
    data['Peptide']= data['Peptide'].apply(lambda x: x.split('.')[1] if x[1]=='.' else x.split('.')[0] if (x[1]!='.' and x[-2]=='.') else x)
    data['count'] = 1
    data = data.groupby(by=['Peptide', 'Protein Accession'], as_index=False).sum()
    data['End'] = data['End']/ data['count']
    data['Start'] = data['Start']/ data['count']
    df_ninf = df_ninf.append(data)
    
df_ninf['count']=1
df_ninf.reset_index(inplace=True, drop=True)

diff_inf=pd.read_excel('difference_data.xlsx', 'inf_diff')
diff_ninf=pd.read_excel('difference_data.xlsx', 'ninf_diff')
diff_inf=pd.merge(diff_inf, df_inf, on=['Peptide', 'Protein Accession'], how='left')
diff_ninf=pd.merge(diff_ninf, df_ninf, on=['Peptide', 'Protein Accession'], how='left')
diff_inf = diff_inf.drop_duplicates('Peptide')
diff_ninf = diff_ninf.drop_duplicates('Peptide')

diff_inf = diff_inf[['Protein Accession','Start','End']]
diff_inf['Protein Accession'] = diff_inf['Protein Accession'].map(lambda x: str(x[3:]) )
diff_inf['Protein Accession'] = diff_inf['Protein Accession'].str[:6]
print(diff_inf)
diff_ninf = diff_ninf[['Protein Accession','Start','End']]
diff_ninf['Protein Accession'] = diff_ninf['Protein Accession'].map(lambda x: str(x[3:]) )
diff_ninf['Protein Accession'] = diff_ninf['Protein Accession'].str[:6]
print(diff_ninf)

    Protein Accession  Start    End
0              P68871  136.0  147.0
3              P68871  137.0  147.0
6              P68871   73.0   87.0
9              P68871    2.0   12.0
12             P69905   72.0   84.0
15             P68871   74.0   87.0
18             P69905  134.0  142.0
21             P68871  139.0  147.0
24             P68871   73.0   86.0
27             P69905  132.0  142.0
30             P69905   85.0   97.0
33             P69905   23.0   33.0
36             P69905   73.0   81.0
39             P69905   74.0   84.0
42             P68871   75.0   87.0
45             P69905   48.0   61.0
48             P69905   35.0   41.0
51             P69905  125.0  142.0
54             P68871    6.0   12.0
57             P69905   68.0   84.0
60             P68871    3.0    9.0
63             P69905   73.0   84.0
66             P68871  140.0  147.0
69             P69905  123.0  133.0
72             P69905  122.0  133.0
75             P68871    3.0   12.0
78             P69905  123.0

### F.2.2 Analysis proteasix

In [160]:
inf_pro = pd.read_excel('output_proteasix.xlsx', 'diff_inf')
inf_pro['Start'] = inf_pro['Start amino acid']
inf_pro['End'] = inf_pro['End amino acid']
inf_pro['Protein Accession'] = inf_pro['Parent Protein AC from input']
inf_pro = inf_pro[['Protease Human/Mouse/Rat', 'Start', 'End', 'Protein Accession']]
ninf_pro = pd.read_excel('output_proteasix.xlsx', 'diff_ninf')
ninf_pro['Start'] = ninf_pro['Start amino acid']
ninf_pro['End'] = ninf_pro['End amino acid']
ninf_pro['Protein Accession'] = ninf_pro['Parent Protein AC from input']
ninf_pro = ninf_pro[['Protease Human/Mouse/Rat', 'Start', 'End', 'Protein Accession']]

df_inf = df_inf.groupby(by=['Peptide','Protein Accession'], as_index=False).sum()
df_inf['End']=df_inf['End']/df_inf['count']
df_inf['Start']=df_inf['Start']/df_inf['count']
df_inf['Protein Accession'] = df_inf['Protein Accession'].map(lambda x: str(x[3:]) )
df_inf['Protein Accession'] = df_inf['Protein Accession'].str[:6]
df_ninf = df_ninf.groupby(by=['Peptide','Protein Accession'], as_index=False).sum()
df_ninf['End']=df_ninf['End']/df_ninf['count']
df_ninf['Start']=df_ninf['Start']/df_ninf['count']
df_ninf['Protein Accession'] = df_ninf['Protein Accession'].map(lambda x: str(x[3:]) )
df_ninf['Protein Accession'] = df_ninf['Protein Accession'].str[:6]

df = pd.merge(df_inf, inf_pro, on=['Protein Accession','Start','End'])
df = df[['Protease Human/Mouse/Rat', 'Start', 'End', '#Feature', 'Peptide']]
df = df.groupby(by= ['Protease Human/Mouse/Rat'], as_index=False).sum()
df = df.sort_values(['#Feature'], ascending=False)
print(df)
df2 = pd.merge(df_ninf, ninf_pro, on=['Protein Accession','Start','End'])
df2 = df2[['Protease Human/Mouse/Rat', 'Start', 'End', '#Feature', 'Peptide']]
df2 = df2.groupby(by= ['Protease Human/Mouse/Rat'], as_index=False).sum()
df2 = df2.sort_values(['#Feature'], ascending=False)
print(df2)

                             Protease Human/Mouse/Rat   Start     End  \
35                         PGA3 (P0DJD8; PEPA3_HUMAN)  5139.0  5832.0   
16  ELANE (P08246; ELNE_HUMAN). Elane (Q3UP87; ELN...  4361.0  4892.0   
14  Ctsl (P06797; CATL1_MOUSE). Ctsl (P07154; CATL...  3837.0  4360.0   
6   Capn1 (O35350; CAN1_MOUSE). CAPN1 (P07384; CAN...  3494.0  3942.0   
7   Capn2 (O08529; CAN2_MOUSE). CAPN2 (P17655; CAN...  3506.0  3954.0   
26  MMP9 (P14780; MMP9_HUMAN). Mmp9 (P41245; MMP9_...  3819.0  4320.0   
15  Ctss (O70370; CATS_MOUSE). CTSS (P25774; CATS_...  2821.0  3234.0   
28  Mep1a (P28825; MEP1A_MOUSE). MEP1A (Q16819; ME...  3205.0  3624.0   
5   CTSG (P08311; CATG_HUMAN). Ctsg (P17977; CATG_...  3340.0  3736.0   
12  Ctsb (P00787; CATB_RAT). CTSB (P07858; CATB_HU...  2556.0  2899.0   
25  MMP7 (P09237; MMP7_HUMAN). Mmp7 (P50280; MMP7_...  3091.0  3486.0   
13  Ctsk (O35186; CATK_RAT). CTSK (P43235; CATK_HU...  2204.0  2541.0   
33  Mmp3 (P03957; MMP3_RAT). MMP3 (P08254; MMP3_HU.

### F.3 5-way venn-diagram

In [ ]:
data1 = pd.read_excel('WF/peptide_WF1.xlsx')
data1 = data1.loc[(data1['#Feature'] >= 4)]
data1['Peptide']=data1['Peptide'].str.replace('[^a-zA-Z]','')
data1 = data1.groupby(['Peptide', 'Accession'], as_index=False).sum()
data1.drop_duplicates(['Peptide'], inplace=True)

data2 = pd.read_excel('WF/peptide_WF2.xlsx')
data2 = data2.loc[(data2['#Feature'] >= 4)]
data2['Peptide']=data2['Peptide'].str.replace('[^a-zA-Z]','')
data2 = data2.groupby(by=['Peptide'], as_index=False).sum()

data3 = pd.read_excel('WF/peptide_WF3.xlsx')
data3 = data3.loc[(data3['#Feature'] >= 4)]
data3['Peptide']=data3['Peptide'].str.replace('[^a-zA-Z]','')
data3 = data3.groupby(by=['Peptide'], as_index=False).sum()

data4 = pd.read_excel('WF/peptide_WF4.xlsx')
data4 = data4.loc[(data4['#Feature'] >= 4)]
data4['Peptide']=data4['Peptide'].str.replace('[^a-zA-Z]','')
data4 = data4.groupby(by=['Peptide'], as_index=False).sum()

data5 = pd.read_excel('WF/peptide_WF5.xlsx')
data5 = data5.loc[(data5['#Feature'] >= 4)]
data5['Peptide']=data5['Peptide'].str.replace('[^a-zA-Z]','')
data5 = data5.groupby(by=['Peptide'], as_index=False).sum()
data1

In [ ]:
data1['Peptide'].to_clipboard(sep='\n', index=False)